# Integrating MindBridge with Databricks: A Step-by-Step Guide

This tutorial will guide you through the process of integrating MindBridge with Databricks. We will walkthrough how to set up an Organization and Engagement using the MindBridge SDK, how to perform a General Ledger Analysis on data from Databricks, then how you can extract key information from your Analysis Results. Here’s what we’ll cover:

1. **Installing the MindBridge SDK in Databricks**  
   We’ll start by installing the MindBridge SDK in your Databricks environment, enabling access to MindBridge’s API features within your notebooks.

2. **Storing your MindBridge API Token in Databricks**  
   Learn to securely save your MindBridge API token in a Databricks-backed secret scope. This step ensures that sensitive credentials are safely managed within Databricks.

3. **Loading your MindBridge API Token in Databricks**  
   With the token stored, we’ll show you how to load and use it to configure your API connection.

4. **Setting Up an Organization and Engagement**  
   Here, you’ll learn how to configure an Organization and Engagement for your Analysis.

5. **Uploading Files to the File Manager**  
    How to upload files into the File Manager created for your Engagement

6. **Creating and Running Analyses**  
   In this section, you’ll learn how to create a new analysis, link the necessary data from the File Manager, and execute the analysis.

7. **Getting the Analysis Results**  
   Finally, we’ll retrieve and display the results of your analysis, providing insights into your data that you can further leverage within Databricks.[](url)


## Installing the MindBridge SDK in Databricks

The following commands can be run to install the [mindbridge-api-python-client](https://pypi.org/project/mindbridge-api-python-client/) to your currently selected cluster. Version 1.5.1 or newer is required for the steps in this tutorial.

In [0]:
%pip install --upgrade mindbridge-api-python-client
dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 478.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Not uninstalling urllib3 at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b29468ee-f01c-4fde-a90f-41582bddb79d
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b29468ee-f01c-4f

## Storing Your MindBridge API Token in Databricks

If you do not have an API Token already you can follow the [Create an API Token Guide](https://support.mindbridge.ai/hc/en-us/articles/9349943782039-Create-an-API-token)
Once that is done, we’ll create a [Databricks-backed secret scope](<https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#create-a-databricks-backed-secret-scope) using the Databricks CLI to securely store your token. If you haven’t set up the Databricks CLI yet, you can follow the setup instructions [here](https://docs.databricks.com/en/dev-tools/cli/index.html).

After installing and configuring the CLI, use the following commands to create a new secret scope and add your API Token to the scope:
<br/><br/>

```bash
# Create the secret scope named "mindbridge-api-tutorials"
databricks secrets create-scope --scope mindbridge-api-tutorials

# Add your MindBridge API Token to the scope
databricks secrets put --scope mindbridge-api-tutorials --key MINDBRIDGE_API_TOKEN --string-value YOUR-API-TOKEN
```
<br/>
Once your token is added, you can verify its existence by running the following:
<br/><br/>

```bash
databricks secrets list-scopes
databricks secrets list --scope mindbridge-api-tutorials
```


## Loading your MindBridge API Token in Databricks

In this section, we’ll load the MindBridge API token securely stored in Databricks and configure the API connection. Replace the ```url``` with the url for your MindBridge instance. Upon execution, you should see details about the user associated with your token.

In [0]:
# Load your token from the secret scope
token = dbutils.secrets.get(scope="mindbridge-api-tutorials", key="MINDBRIDGE_API_TOKEN")

# Create a connection to the server
import mindbridgeapi as mbapi
url = "yoursubdomain.mindbridge.ai"
server = mbapi.Server(url=url, token=token)

# Get the current user
user = server.users.get_current()
print(user)

created_user_info=ApiUserInfoRead(user_id='65d4e4091b189d63add5f0fc', user_name='Owen Allen') creation_date=datetime.datetime(2024, 9, 20, 17, 2, 52, 711000, tzinfo=TzInfo(UTC)) email=None enabled=True first_name="Owen's Token" id='66edaabc0a1348596f88dfbf' last_modified_date=datetime.datetime(2024, 11, 4, 18, 57, 39, 88000, tzinfo=TzInfo(UTC)) last_modified_user_info=ApiUserInfoRead(user_id='65d4e4091b189d63add5f0fc', user_name='Owen Allen') last_name='' role=<Role.ROLE_ADMIN: 'ROLE_ADMIN'> service_account=True validated=True version=10


## Setting up an Organization and Engagement

In this section, we’ll walk through setting up an Organization and an Engagement to house your Analysis. We’ll set up the Engagement using the MindBridge for-profit library. If you already have an existing Organization and Engagement you would like to use, you can update the ```organization_name``` and ```engagement_name``` values to use your preferred entities. If your Engagement uses a different Library, you may need to update later steps so that your Analysis Type and Analysis Source Types are compatible with the Library you selected.

In [0]:
# Define details for organization, engagement, and library
organization_name = "MindBridge Databricks Integration"
engagement_name = "My Engagement"
library_name = "MindBridge for-profit"

# Step 1: Create the Organization, or get it if it already exists
try:
    organization_item = mbapi.OrganizationItem(
        name=organization_name,
        external_client_code="My Client ID",  # Optional
        manager_user_ids=[user.id]            # Optional
    )
    organization = server.organizations.create(organization_item)
    print(f"Created the Organization: '{organization_name}'")
except mbapi.exceptions.ValidationError:
    organization = next(server.organizations.get({"name": organization_name}))
    print(f"Organization '{organization_name}' already exists. Fetched it instead.")

# Step 2: Get the Library we want to use in the Engagement
system_libraries = server.libraries.get({"system": True})
mindbridge_for_profit_library = next((x for x in system_libraries if x.name == library_name), None)

# Step 3: Create the Engagement, or get it if it already exists
try:
    engagement_item = mbapi.EngagementItem(
        organization_id=organization.id,
        name=engagement_name,
        engagement_lead_id=user.id,
        library_id=mindbridge_for_profit_library.id
    )
    engagement = server.engagements.create(engagement_item)
    print(f"Created the Engagement: '{engagement_name}'")
except mbapi.exceptions.ValidationError:
    engagement = next(server.engagements.get({"organizationId": organization.id, "name": engagement_name}))
    print(f"Engagement '{engagement_name}' already exists within the Organization. Fetched it instead.")

Created the Organization: 'MindBridge Databricks Integration'
Created the Engagement: 'My Engagement'


## Uploading Files to the File Manager

After an Engagement is created, a File Manager entity is automatically set up to store data used by Analyses within the Engagement. We will upload our example General Ledger file from the data folder to the File Manager.

In [0]:
import os

relative_path = './data/GENERAL_LEDGER_JOURNAL.csv'
full_path = os.path.abspath(relative_path)

gl_file_manager_item = mbapi.FileManagerItem(engagement_id=engagement.id)

# Upload General Ledger
gl_file_manager_file = server.file_manager.upload(
    input_item=gl_file_manager_item,
    input_file=full_path
)

# We can check to see if our files are in the File Manager
file_manager_generator = server.file_manager.get({"engagementId": engagement.id})
print("Here are the files in the File Manager for your Engagement")
for file_manager_entity in file_manager_generator:
  print(file_manager_entity.original_name)

Here are the files in the File Manager for your Engagement
GENERAL_LEDGER_JOURNAL.csv


## Creating and Running Analyses

With our Engagement set up and files uploaded, we’re ready to create an Analysis. We’ll import the General Ledger File Manager as an Analysis Source, mark the Account Mappings as verified then run the Analysis.

If you used a custom library instead of the MindBridge for-profit library used in *Setting up an Organization and Engagement*, you will need to upload a Chart of Accounts or use the Account Mappings endpoints to map the accounts within the General Ledger. In this example we will use the Verify Mappings endpoint to use the suggested Account Mappings created by MindBridge.

In [0]:
import time

# Create an analysis
analysis_item = mbapi.AnalysisItem(
    engagement_id=engagement.id,
    analysis_periods=[{"startDate": "2020-01-01", "endDate": "2021-01-01"}],
    analysis_type_id=mbapi.AnalysisTypeItem.GENERAL_LEDGER,
    currency_code="CAD",
    name="General Ledger Analysis"
)

analysis = server.analyses.create(analysis_item)

# Add the General Ledger from the File Manager as an Analysis Source
gl_analysis_source_item = mbapi.AnalysisSourceItem(
    engagement_id=engagement.id,
    analysis_id=analysis.id,
    file_manager_file_id=gl_file_manager_file.id,
    analysis_period_id=analysis.analysis_periods[0].id,
    analysis_source_type_id=mbapi.AnalysisSourceTypeItem.GENERAL_LEDGER_JOURNAL,
    target_workflow_state=mbapi.TargetWorkflowState.COMPLETED,
)

print("Creating Analysis Source")
gl_analysis_source = server.analysis_sources.create(gl_analysis_source_item)

print("Waiting for Analysis Source to be ready")
max_polls = 5 * 60
polls = 0
prev_state = ""
while polls < max_polls:
    polls += 1
    time.sleep(1)
    gl_analysis_source = server.analysis_sources.get_by_id(gl_analysis_source.id)
    if gl_analysis_source.workflow_state.value != prev_state:
        print(f"Current State: {gl_analysis_source.workflow_state.value}")
        prev_state = gl_analysis_source.workflow_state.value
    if gl_analysis_source.workflow_state.value == gl_analysis_source_item.target_workflow_state.value:
        print("Analysis Source is ready.")
        break

print("Setting Account Mappings to Verified")
engagement = server.engagements.verify_account_mappings(engagement)
analysis = server.analyses.wait_for_analysis_sources(analysis)

print("Running the Analysis")
analysis = server.analyses.run(analysis)
print("Analysis has been started")

analysis = server.analyses.wait_for_analysis(analysis)
print("Analysis is complete")

Creating Analysis Source
Waiting for Analysis Source to be ready
Current State: ANALYZING_COLUMNS
Current State: ANALYZING_EFFECTIVE_DATE_METRICS
Current State: CHECKING_INTEGRITY
Current State: PARSING
Current State: COMPLETED
Analysis Source is ready.
Setting Account Mappings to Verified
Running the Analysis
Analysis has been started
Analysis is complete


## Getting the Analysis Results


### Overview of Data Table information
First we will take a look at what Data Tables exist on this Analysis and how the data is structured. Below is some code to generate an overview of each Data Table in the Analysis, including the fields, data type and whether the field is searchable.

In [0]:

import pandas as pd

server.analyses.restart_data_tables(analysis)

# Loop through each data_table and display information
for data_table in analysis.data_tables:
    # Display general data_table information
    general_info = pd.DataFrame({
        "Analysis ID": [data_table.analysis_id],
        "Data Table ID": [data_table.id],
        "Logical Name": [data_table.logical_name],
        "Type": [data_table.type]
    })
    display(general_info)
    
    # Prepare column data for this data_table
    column_data = [
        {
            "Field": col.field,
            "Filter Only": col.filter_only,
            "Keyword Search": col.keyword_search,
            "Type": col.type.value
        }
        for col in data_table.columns
    ]
    
    # Create DataFrame for column data and display it
    column_df = pd.DataFrame(column_data)
    display(column_df)


Analysis ID,Data Table ID,Logical Name,Type
67293ab73991ad5a982c0fc7,67293b3aef57c21ed1142ea7,flows_compact_67293acc3991ad5a982c0fd2,flows_compact


Field,Filter Only,Keyword Search,Type
flow_id,false,false,INT64
txid,false,false,INT64
src_rowid,false,false,INT64
dest_rowid,false,false,INT64
source_account_num,false,false,STRING
dest_account_num,false,false,STRING
amount,false,false,INT64


Analysis ID,Data Table ID,Logical Name,Type
67293ab73991ad5a982c0fc7,67293b4cc5da1d4dd0a2338f,gl_journal_lines,null


Field,Filter Only,Keyword Search,Type
rowid,false,false,INT64
txid,false,false,INT64
credit,false,true,MONEY_100
debit,false,true,MONEY_100
effective_date,false,false,DATE
entered_date,false,false,DATE
transaction,false,true,STRING
account,false,true,STRING
unmapped_balance,false,true,STRING
user_id,false,true,STRING


Analysis ID,Data Table ID,Logical Name,Type
67293ab73991ad5a982c0fc7,67293b4cc5da1d4dd0a23390,gl_journal_tx,null


Field,Filter Only,Keyword Search,Type
txid,false,false,INT64
effective_date,false,true,DATE
debit,false,true,MONEY_100
credit,false,true,MONEY_100
entries_count,false,false,INT32
monetary_value,false,false,MONEY_100
transaction,false,true,STRING
accounts,false,true,ARRAY_STRINGS
entered_date,false,true,DATE
type,false,true,STRING


### Extracting Key Info from MindBridge 

Now let's extract specific data from our Data Tables. In this example we will fetch all of the transactions which had a Risk Score greater or equal to 30%. Then we will save the results in a CSV and load them into a pandas DataFrame.

In [0]:
import os
# Define the output file path where the results will be saved

output_relative_path = './results/result.csv'
output_full_path = os.path.abspath(output_relative_path)

print("Requesting Elevated Risk General Ledger Transactions")
# Restart data tables to reset generator
server.analyses.restart_data_tables(analysis)

# Select the transactions table
data_table = next(x for x in analysis.data_tables if x.logical_name == "gl_journal_tx")

# Define a query to extract transactions with a risk score greater than or equal to 30%
query = {"risk": {"$gte": 3000}}

# Export the filtered data
export_async_result = server.data_tables.export(data_table, query=query)
server.data_tables.wait_for_export(export_async_result)

# Define the output path for saving the CSV file
path_output = server.data_tables.download(
    export_async_result,
    output_file_path=output_full_path,
)
print(f"Success! Saved to: {path_output}")


/Workspace/Users/owen.allen@mindbridge.ai/mindbridge-databricks-notebooks/results/result.csv
Requesting Elevated Risk General Ledger Transactions


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `int` but got `MindBridgeQueryTerm4` with value `MindBridgeQueryTerm4(field_gte=3000)` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `float` but got `MindBridgeQueryTerm4` with value `MindBridgeQueryTerm4(field_gte=3000)` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `bool` but got `MindBridgeQueryTerm4` with value `MindBridgeQueryTerm4(field_gte=3000)` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `str` but got `MindBridgeQueryTerm4` with value `MindBridgeQueryTerm4(field_gte=3000)` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Success! Saved to: /Workspace/Users/owen.allen@mindbridge.ai/mindbridge-databricks-notebooks/results/result.csv


In [0]:
# Read the CSV file saved in the previous step in pandas

df = pd.read_csv(output_full_path)

display(df)

txid,effective_date,debit,credit,entries_count,monetary_value,transaction,accounts,entered_date,type,user_id,risk,cp_failed_journal_entry_weekend_post,cp_failed_group_by_trendshock,cp_failed_journal_entry_last_3_digits,cp_failed_journal_entry_complex_structure_v2,cp_failed_prior_period_frequency,cp_failed_journal_entry_manual_entry,cp_failed_journal_entry_is_reversal_of,cp_failed_journal_entry_gap_edge,cp_failed_journal_entry_zero,cp_failed_journal_entry_flat_gifi_bucket_rarity,cp_failed_journal_entry_reporting_analysis_period_adjustment,cp_failed_journal_entry_flat_sos_v2,cp_failed_journal_entry_unbalanced_transaction,cp_failed_journal_entry_cash_out_the_door,cp_failed_journal_entry_cash_to_bad_debt,cp_failed_journal_entry_flat_expert_flow_v2,cp_failed_journal_entry_is_reversed_by,cp_failed_journal_entry_start_of_period,cp_failed_journal_entry_high_dollar_value,cp_failed_journal_entry_nfp_expense_flurry,cp_failed_journal_entry_material_value,cp_failed_journal_entry_suspicious_keyword,cp_failed_journal_entry_start_of_year,cp_failed_journal_entry_reporting_adjustment,cp_failed_journal_entry_complex_instrument,cp_failed_journal_entry_two_digit_benford_v2,cp_failed_prior_period_overall_score_change,cp_failed_markov_chain,cp_failed_prior_period_dollar_change,cp_failed_journal_entry_end_of_year,cp_failed_journal_entry_empty_text_field,cp_failed_journal_entry_isolation_forest,cp_failed_journal_entry_flat_unusual_amount_v4,cp_failed_journal_entry_flow_analysis,cp_failed_journal_entry_duplicate_v2,cp_failed_journal_entry_end_of_period,unknown_flow,cp_score_journal_entry_weekend_post,cp_score_group_by_trendshock,cp_score_journal_entry_last_3_digits,cp_score_journal_entry_complex_structure_v2,cp_score_prior_period_frequency,cp_score_journal_entry_manual_entry,cp_score_journal_entry_is_reversal_of,cp_score_journal_entry_gap_edge,cp_score_journal_entry_zero,cp_score_journal_entry_flat_gifi_bucket_rarity,cp_score_journal_entry_reporting_analysis_period_adjustment,cp_score_journal_entry_flat_sos_v2,cp_score_journal_entry_unbalanced_transaction,cp_score_journal_entry_cash_out_the_door,cp_score_journal_entry_cash_to_bad_debt,cp_score_journal_entry_flat_expert_flow_v2,cp_score_journal_entry_is_reversed_by,cp_score_journal_entry_start_of_period,cp_score_journal_entry_high_dollar_value,cp_score_journal_entry_nfp_expense_flurry,cp_score_journal_entry_material_value,cp_score_journal_entry_suspicious_keyword,cp_score_journal_entry_start_of_year,cp_score_journal_entry_reporting_adjustment,cp_score_journal_entry_complex_instrument,cp_score_journal_entry_two_digit_benford_v2,cp_score_prior_period_overall_score_change,cp_score_markov_chain,cp_score_prior_period_dollar_change,cp_score_journal_entry_end_of_year,cp_score_journal_entry_empty_text_field,cp_score_journal_entry_isolation_forest,cp_score_journal_entry_flat_unusual_amount_v4,cp_score_journal_entry_flow_analysis,cp_score_journal_entry_duplicate_v2,cp_score_journal_entry_end_of_period,bs_delta,is_delta,account_tags_increasing_expense,account_tags_increasing_asset,account_tags_increasing_ar,account_tags_increasing_liabilities,account_tags_increasing_income,account_tags_increasing_cash,account_tags_increasing_inventory,account_tags_increasing_ebitda,account_tags_increasing_contra,account_tags_increasing_ap,account_tags_decreasing_expense,account_tags_decreasing_asset,account_tags_decreasing_ar,account_tags_decreasing_liabilities,account_tags_decreasing_income,account_tags_decreasing_cash,account_tags_decreasing_inventory,account_tags_decreasing_ebitda,account_tags_decreasing_contra,account_tags_decreasing_ap,account_hierarchy_codes
137438953674,2020-01-02,209070,209070,2,209070,211,'14012000';'54054000',2020-01-02,EX,Susan Farrow - AP Clerk,3026,false,false,false,false,false,false,false,false,false,true,false,false,false,true,false,false,false,true,true,false,true,false,true,false,false,false,false,true,false,false,false,true,false,false,false,false,false,0,0,0,0,0,0,0,0,0,6406,0,3107,0,10000,0,100